# Testing Collaborative Filtering Recommender System

In [1]:
import pandas as pd
import numpy as np
import joblib
from IPython.display import HTML

In [2]:
cosine_sim2_loaded, dist_euc_loaded, df_2_loaded = joblib.load('collaborative_recommender_model.pkl')

In [3]:
def find_similar_wines_combined(wines, count=10):
    wines_cosine = [wine for wine in wines if wine in cosine_sim2_loaded.columns]
    wines_summed_cosine = cosine_sim2_loaded[wines_cosine].apply(lambda row: np.sum(row), axis=1)
    wines_summed_cosine = wines_summed_cosine.sort_values(ascending=False)
    ranked_wines_cosine = wines_summed_cosine.index[wines_summed_cosine.index.isin(wines_cosine) == False]
    ranked_wines_cosine = ranked_wines_cosine.tolist()
    ranked_wines_cosine = ranked_wines_cosine[:count]

    wines_euclidean = [wine for wine in wines if wine in dist_euc_loaded.columns]
    wines_summed_euclidean = dist_euc_loaded[wines_euclidean].apply(lambda row: np.sum(row), axis=1)
    wines_summed_euclidean = wines_summed_euclidean.sort_values()
    ranked_wines_euclidean = wines_summed_euclidean.index[wines_summed_euclidean.index.isin(wines_euclidean) == False]
    ranked_wines_euclidean = ranked_wines_euclidean.tolist()
    ranked_wines_euclidean = ranked_wines_euclidean[:count]

    combined_ranking = {}
    for i, wine in enumerate(ranked_wines_cosine):
        combined_ranking[wine] = combined_ranking.get(wine, 0) + i

    for i, wine in enumerate(ranked_wines_euclidean):
        combined_ranking[wine] = combined_ranking.get(wine, 0) + i

    sorted_combined_ranking = sorted(combined_ranking.items(), key=lambda x: x[1])

    top_10_wine_indices = [df_2_loaded.index[df_2_loaded['WineName'] == wine].tolist()[0] for wine, _ in sorted_combined_ranking[:count]]
    top_10_wines = df_2_loaded.loc[top_10_wine_indices, ['WineName', 'Website']].copy()

    top_10_wines['Website'] = top_10_wines['Website'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

    display(HTML(top_10_wines.to_html(escape=False, index=False)))

In [4]:
find_similar_wines_combined(['Vecchia Grappa Moscato'], 10)

WineName,Website
STORIA Grappa Merlot,casavalduga.com.br
Cave Cabernet Sauvignon,luizargenta.com.br
Garnacha Rosé,tesco.com
A Rosé Is A Rosé Is A Rosé,luigibosca.com.ar
Grüner Veltliner Alte Reben,urbanihof.at
Perlé Nero,cantineferrari.it
Fragolino,bautistamarti.com
Reserva Chardonnay,fabremontmayou.com
Trollinger Lemberger Trocken,rolf-willy.de
Colheita Tardia Malvasia Bianca,vinicolaaurora.com.br
